In [1]:
import numpy as np
import cv2
from skimage.exposure import rescale_intensity
import os

In [2]:
kernal_size = [3,3]
padding = kernal_size[0]//2


avaliable = ['baggage_claim.jpg','dining_table.jpg','living_room.jpg','soccer.jpg']
image_directory = '/Users/sanjaymsanthosh/Desktop/projects/project 2019/human detection/yolo/yolo-object-detection/images/'
image_location = os.path.join(image_directory,avaliable[1])

In [3]:
def convolve(image, kernel):
    # Get the dimensions of the image and kernel
    (img_h, img_w) = image.shape[:2]
    (ker_h, ker_w) = kernel.shape[:2]

    # Allocate memory of the output image, ensuring to 'pad' the borders of the input image so the spacial size
    # (i.e. width and height) are not reduced
    pad = (ker_w - 1) // 2
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    output = np.zeros((img_h, img_w), dtype='float')

    # Loop over the input image, 'sliding' the kernel across each (x, y) coordinate from left-to-right and
    # top-to-bottom
    for y in np.arange(pad, img_h + pad):
        for x in np.arange(pad, img_w + pad):
            # Extract the ROI of the image by extracting the 'center' region of the current (x, y) coordinates
            # dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]

            # Perform the actual convolution
            k = (roi * kernel).sum()

            # Store the convolved value in the (x, y) coordinate of the output image
            output[y - pad, x - pad] = k

    # Rescale the output image to be in the range [0, 255]
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype('uint8')

    # Return the output image
    return output


In [4]:
small_blur = np.ones((7, 7), dtype='float') * (1.0 / (7 * 7))
large_blur = np.ones((21, 21), dtype='float') * (1.0 / (21 * 21))

# Construct a sharpening filter
sharpen = np.array(([0, -1, 0],
                    [-1, 5, -1],
                    [0, -1, 0]), dtype='float')

# Construct the Laplacian kernel used to detect edge-like regions of an image
laplacian = np.array(([0, 1, 0],
                      [1, -4, 1],
                      [0, 1, 0]), dtype='int')

# Construct the Sobel x-axis kernel
sobel_x = np.array(([-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]), dtype='int')

# Construct the Sobel y-axis kernel
sobel_y = np.array(([-1, -2, -1],
                    [0, 0, 0],
                    [1, 2, 1]), dtype='int')

# Construct an emboss kernel
emboss = np.array(([-2, -1, 0],
                   [-1, 1, 1],
                   [0, 1, 2]), dtype='int')

In [5]:
kernel_bank = (('small_blur', small_blur),
               ('large_blur', large_blur),
               ('sharpen', sharpen),
               ('laplacian', laplacian),
               ('sobel_x', sobel_x),
               ('sobel_y', sobel_y),
               ('emboss', emboss))

In [6]:
image = cv2.imread(image_location)
grey = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)



In [7]:
for (kernel_name, kernel) in kernel_bank:
    # Apply the kernel to the greyscale image using both 'convolve' functions
    print('[INFO]: Applying {} kernel'.format(kernel_name))
    convolve_output = convolve(grey, kernel)
    opencv_output = cv2.filter2D(grey, -1, kernel)

    # Show the output image
    cv2.imshow('Original', grey)
    cv2.imshow('{} - convolve'.format(kernel_name), convolve_output)
    cv2.imshow('{} - filter2D'.format(kernel_name), opencv_output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[INFO]: Applying small_blur kernel
[INFO]: Applying large_blur kernel
[INFO]: Applying sharpen kernel
[INFO]: Applying laplacian kernel
[INFO]: Applying sobel_x kernel
[INFO]: Applying sobel_y kernel
[INFO]: Applying emboss kernel
